## CorEx

In [82]:
"""CorEx Hierarchical Topic Models
Use the principle of Total Cor-relation Explanation (CorEx) to construct
hierarchical topic models. This module is specially designed for sparse count
data and implements semi-supervision via the information bottleneck.
Greg Ver Steeg and Aram Galstyan. "Maximally Informative Hierarchical
Representations of High-Dimensional Data." AISTATS, 2015.
Gallagher et al. "Anchored Correlation Explanation: Topic Modeling with Minimal
Domain Knowledge." TACL, 2017.
Code below written by:
Greg Ver Steeg (gregv@isi.edu)
Ryan J. Gallagher
David Kale
Lily Fierro
License: Apache V2
"""

import numpy as np  # Tested with 1.8.0
from os import makedirs
from os import path
from scipy.special import logsumexp # Tested with 0.13.0
import scipy.sparse as ss
from six import string_types # For Python 2&3 compatible string checking
from sklearn.externals import joblib


class Corex(object):
    """
    Anchored CorEx hierarchical topic models
    Code follows sklearn naming/style (e.g. fit(X) to train)
    Parameters
    ----------
    n_hidden : int, optional, default=2
        Number of hidden units.
    max_iter : int, optional
        Maximum number of iterations before ending.
    verbose : int, optional
        The verbosity level. The default, zero, means silent mode. 1 outputs TC(X;Y) as you go
        2 output alpha matrix and MIs as you go.
    tree : bool, default=True
        In a tree model, each word can only appear in one topic. tree=False is not yet implemented.
    count : string, {'binarize', 'fraction'}
        Whether to treat counts (>1) by directly binarizing them, or by constructing a fractional count in [0,1].
    seed : integer or numpy.RandomState, optional
        A random number generator instance to define the state of the
        random permutations generator. If an integer is given, it fixes the
        seed. Defaults to the global numpy random number generator.
    Attributes
    ----------
    labels : array, [n_samples, n_hidden]
        Label for each hidden unit for each sample.
    clusters : array, [n_visible]
        Cluster label for each input variable.
    p_y_given_x : array, [n_samples, n_hidden]
        p(y_j=1|x) for each sample.
    alpha : array-like, shape [n_hidden, n_visible]
        Adjacency matrix between input variables and hidden units. In range [0,1].
    mis : array, [n_hidden, n_visible]
        Mutual information between each (visible/observed) variable and hidden unit
    tcs : array, [n_hidden]
        TC(X_Gj;Y_j) for each hidden unit
    tc : float
        Convenience variable = Sum_j tcs[j]
    tc_history : array
        Shows value of TC over the course of learning. Hopefully, it is converging.
    words : list of strings
        Feature names that label the corresponding columns of X
    References
    ----------
    [1]     Greg Ver Steeg and Aram Galstyan. "Discovering Structure in
            High-Dimensional Data Through Correlation Explanation."
            NIPS, 2014. arXiv preprint arXiv:1406.1222.
    [2]     Greg Ver Steeg and Aram Galstyan. "Maximally Informative
            Hierarchical Representations of High-Dimensional Data"
            AISTATS, 2015. arXiv preprint arXiv:1410.7404.
    """

    def __init__(self, n_hidden=2, max_iter=200, eps=1e-5, seed=None, verbose=False, count='binarize',
                 tree=True, **kwargs):
        self.n_hidden = n_hidden  # Number of hidden factors to use (Y_1,...Y_m) in paper
        self.max_iter = max_iter  # Maximum number of updates to run, regardless of convergence
        self.eps = eps  # Change to signal convergence
        self.tree = tree
        np.random.seed(seed)  # Set seed for deterministic results
        self.verbose = verbose
        self.t = 20  # Initial softness of the soft-max function for alpha (see NIPS paper [1])
        self.count = count  # Which strategy, if necessary, for binarizing count data
        if verbose > 0:
            np.set_printoptions(precision=3, suppress=True, linewidth=200)
            print('corex, rep size:', n_hidden)
        if verbose:
            np.seterr(all='warn')
            # Can change to 'raise' if you are worried to see where the errors are
            # Locally, I "ignore" underflow errors in logsumexp that appear innocuous (probabilities near 0)
        else:
            np.seterr(all='ignore')

    def label(self, p_y_given_x):
        """Maximum likelihood labels for some distribution over y's"""
        return (p_y_given_x > 0.5).astype(bool)

    @property
    def labels(self):
        """Maximum likelihood labels for training data. Can access with self.labels (no parens needed)"""
        return self.label(self.p_y_given_x)

    @property
    def clusters(self):
        """Return cluster labels for variables"""
        return np.argmax(self.alpha, axis=0)

    @property
    def sign(self):
        """Return the direction of correlation, positive or negative, for each variable-latent factor."""
        return np.sign(self.theta[3] - self.theta[2]).T

    @property
    def tc(self):
        """The total correlation explained by all the Y's.
        """
        return np.sum(self.tcs)

    def fit(self, X, anchors=None, anchor_strength=1, words=None, docs=None):
        """
        Fit CorEx on the data X. See fit_transform.
        """
        self.fit_transform(X, anchors=anchors, anchor_strength=anchor_strength, words=words, docs=docs)
        return self

    def fit_transform(self, X, anchors=None, anchor_strength=1, words=None, docs=None):
        """Fit CorEx on the data
        Parameters
        ----------
        X : scipy sparse CSR or a numpy matrix, shape = [n_samples, n_visible]
            Count data or some other sparse binary data.
        anchors : A list of variables anchor each corresponding latent factor to.
        anchor_strength : How strongly to weight the anchors.
        words : list of strings that label the corresponding columns of X
        docs : list of strings that label the corresponding rows of X
        Returns
        -------
        Y: array-like, shape = [n_samples, n_hidden]
           Learned values for each latent factor for each sample.
           Y's are sorted so that Y_1 explains most correlation, etc.
        """
        X = self.preprocess(X)
        self.initialize_parameters(X, words, docs)
        if anchors is not None:
            anchors = self.preprocess_anchors(list(anchors))
        p_y_given_x = np.random.random((self.n_samples, self.n_hidden))
        if anchors is not None:
            for j, a in enumerate(anchors):
                p_y_given_x[:, j] = 0.5 * p_y_given_x[:, j] + 0.5 * X[:, a].mean(axis=1).A1  # Assumes X is a binary matrix

        for nloop in range(self.max_iter):
            if nloop > 1:
                for j in range(self.n_hidden):
                    if self.sign[j, np.argmax(self.mis[j])] < 0:
                        # Switch label for Y_j so that it is correlated with the top word
                        p_y_given_x[:, j] = 1. - p_y_given_x[:, j]
            self.log_p_y = self.calculate_p_y(p_y_given_x)
            self.theta = self.calculate_theta(X, p_y_given_x, self.log_p_y)  # log p(x_i=1|y)  nv by m by k

            if nloop > 0:  # Structure learning step
                self.alpha = self.calculate_alpha(X, p_y_given_x, self.theta, self.log_p_y, self.tcs)
            if anchors is not None:
                for a in flatten(anchors):
                    self.alpha[:, a] = 0
                for ia, a in enumerate(anchors):
                    self.alpha[ia, a] = anchor_strength

            p_y_given_x, _, log_z = self.calculate_latent(X, self.theta)

            self.update_tc(log_z)  # Calculate TC and record history to check convergence
            self.print_verbose()
            if self.convergence():
                break

        if self.verbose:
            print('Overall tc:', self.tc)

        if anchors is None:
            self.sort_and_output(X)
        self.p_y_given_x, self.log_p_y_given_x, self.log_z = self.calculate_latent(X, self.theta)  # Needed to output labels
        self.mis = self.calculate_mis(self.theta, self.log_p_y)  # / self.h_x  # could normalize MIs
        return self.labels

    def transform(self, X, details=False):
        """
        Label hidden factors for (possibly previously unseen) samples of data.
        Parameters: samples of data, X, shape = [n_samples, n_visible]
        Returns: , shape = [n_samples, n_hidden]
        """
        X = self.preprocess(X)
        p_y_given_x, _, log_z = self.calculate_latent(X, self.theta)
        labels = self.label(p_y_given_x)
        if details == 'surprise':
            # TODO: update
            # Totally experimental
            n_samples = X.shape[0]
            alpha = np.zeros((self.n_hidden, self.n_visible))
            for i in range(self.n_visible):
                alpha[np.argmax(self.alpha[:, i]), i] = 1
            log_p = np.empty((2, n_samples, self.n_hidden))
            c0 = np.einsum('ji,ij->j', alpha, self.theta[0])
            c1 = np.einsum('ji,ij->j', alpha, self.theta[1])  # length n_hidden
            info0 = np.einsum('ji,ij->ij', alpha, self.theta[2] - self.theta[0])
            info1 = np.einsum('ji,ij->ij', alpha, self.theta[3] - self.theta[1])
            log_p[1] = c1 + X.dot(info1)  # sum_i log p(xi=xi^l|y_j=1)  # Shape is 2 by l by j
            log_p[0] = c0 + X.dot(info0)  # sum_i log p(xi=xi^l|y_j=0)
            surprise = [-np.sum([log_p[labels[l, j], l, j] for j in range(self.n_hidden)]) for l in range(n_samples)]
            return p_y_given_x, log_z, np.array(surprise)
        elif details:
            return p_y_given_x, log_z
        else:
            return labels

    def predict_proba(self, X):
        return self.transform(X, details=True)

    def predict(self, X):
        return self.transform(X, details=False)

    def preprocess(self, X):
        """Data can be binary or can be in the range [0,1], where that is interpreted as the probability to
        see this variable in a given sample"""
        if X.max() > 1:
            if self.count == 'binarize':
                X = (X > 0)
            elif self.count == 'fraction':
                X = X.astype(float)
                count = np.array(X.sum(axis=0), dtype=float).ravel()
                length = np.array(X.sum(axis=1)).ravel().clip(1)
                bg_rate = ss.diags(float(X.sum()) / count, 0)
                doc_length = ss.diags(1. / length, 0)
                # max_counts = ss.diags(1. / X.max(axis=1).A.ravel(), 0)
                X = doc_length * X * bg_rate
                X.data = np.clip(X.data, 0, 1)  # np.log(X.data) / (np.log(X.data) + 1)
        return X

    def initialize_parameters(self, X, words, docs):
        """Store some statistics about X for future use, and initialize alpha, tc"""
        self.n_samples, self.n_visible = X.shape
        if self.n_hidden > 1:
            self.alpha = np.random.random((self.n_hidden, self.n_visible))
            # self.alpha /= np.sum(self.alpha, axis=0, keepdims=True)
        else:
            self.alpha = np.ones((self.n_hidden, self.n_visible), dtype=float)
        self.tc_history = []
        self.tcs = np.zeros(self.n_hidden)
        self.word_counts = np.array(
            X.sum(axis=0)).ravel()  # 1-d array of total word occurrences. (Probably slow for CSR)
        if np.any(self.word_counts == 0) or np.any(self.word_counts == self.n_samples):
            print('WARNING: Some words never appear (or always appear)')
            self.word_counts = self.word_counts.clip(0.01, self.n_samples - 0.01)
        self.word_freq = (self.word_counts).astype(float) / self.n_samples
        self.px_frac = (np.log1p(-self.word_freq) - np.log(self.word_freq)).reshape((-1, 1))  # nv by 1
        self.lp0 = np.log1p(-self.word_freq).reshape((-1, 1))  # log p(x_i=0)
        self.h_x = binary_entropy(self.word_freq)
        if self.verbose:
            print('word counts', self.word_counts)
        # Set column labels
        self.words = words
        if words is not None:
            if len(words) != X.shape[1]:
                print('WARNING: number of column labels != number of columns of X. Check len(words) and X.shape[1]')
            col_index2word = {index:word for index,word in enumerate(words)}
            word2col_index = {word:index for index,word in enumerate(words)}
            self.col_index2word = col_index2word
            self.word2col_index = word2col_index
        else:
            self.col_index2word = None
            self.word2col_index = None
        # Set row labels
        self.docs = docs
        if docs is not None:
            if len(docs) != X.shape[0]:
                print('WARNING: number of row labels != number of rows of X. Check len(docs) and X.shape[0]')
            row_index2doc = {index:doc for index,doc in enumerate(docs)}
            self.row_index2doc = row_index2doc
        else:
            self.row_index2doc = None

    def update_word_parameters(self, X, words):
        """
        updates parameters that need to be changed for each new model update
        specifically, this re-calculates word count related parameters to be based on X,
        where X is a batch of new data
        """
        self.n_samples, self.n_visible = X.shape
        self.word_counts = np.array(
            X.sum(axis=0)).ravel()  # 1-d array of total word occurrences. (Probably slow for CSR)
        if np.any(self.word_counts == 0) or np.any(self.word_counts == self.n_samples):
            print('WARNING: Some words never appear (or always appear)')
            self.word_counts = self.word_counts.clip(0.01, self.n_samples - 0.01)
        self.word_freq = (self.word_counts).astype(float) / self.n_samples
        self.px_frac = (np.log1p(-self.word_freq) - np.log(self.word_freq)).reshape((-1, 1))  # nv by 1
        self.lp0 = np.log1p(-self.word_freq).reshape((-1, 1))  # log p(x_i=0)
        self.h_x = binary_entropy(self.word_freq)
        if self.verbose:
            print('word counts', self.word_counts)
        self.words = words
        if words is not None:
            if len(words) != X.shape[1]:
                print('WARNING: number of column labels != number of columns of X. Check len(words) and X.shape[1]')
            col_index2word = {index:word for index,word in enumerate(words)}
            word2col_index = {word:index for index,word in enumerate(words)}
            self.col_index2word = col_index2word
            self.word2col_index = word2col_index
        else:
            self.col_index2word = None
            self.word2col_index = None

    def preprocess_anchors(self, anchors):
        """Preprocess anchors so that it is a list of column indices if not already"""
        if anchors is not None:
            for n, anchor_list in enumerate(anchors):
                # Check if list of anchors or a single str or int anchor
                if type(anchor_list) is not list:
                    anchor_list = [anchor_list]
                # Convert list of anchors to list of anchor indices
                new_anchor_list = []
                for anchor in anchor_list:
                    # Turn string anchors into index anchors
                    if isinstance(anchor, string_types):
                        if self.words is not None:
                            if anchor in self.word2col_index:
                                new_anchor_list.append(self.word2col_index[anchor])
                            else:
                                raise KeyError('Anchor word not in word column labels provided to CorEx: {}'.format(anchor))
                        else:
                                raise NameError("Provided non-index anchors to CorEx without also providing 'words'")
                    else:
                        new_anchor_list.append(anchor)
                # Update anchors with new anchor list
                if len(new_anchor_list) == 1:
                    anchors[n] = new_anchor_list[0]
                else:
                    anchors[n] = new_anchor_list

        return anchors

    def calculate_p_y(self, p_y_given_x):
        """Estimate log p(y_j=1)."""
        return np.log(np.mean(p_y_given_x, axis=0))  # n_hidden, log p(y_j=1)

    def calculate_theta(self, X, p_y_given_x, log_p_y):
        """Estimate marginal parameters from data and expected latent labels."""
        # log p(x_i=1|y)
        n_samples = X.shape[0]
        p_dot_y = X.T.dot(p_y_given_x).clip(0.01 * np.exp(log_p_y), (n_samples - 0.01) * np.exp(
            log_p_y))  # nv by ns dot ns by m -> nv by m  # TODO: Change to CSC for speed?
        lp_1g1 = np.log(p_dot_y) - np.log(n_samples) - log_p_y
        lp_1g0 = np.log(self.word_counts[:, np.newaxis] - p_dot_y) - np.log(n_samples) - log_1mp(log_p_y)
        lp_0g0 = log_1mp(lp_1g0)
        lp_0g1 = log_1mp(lp_1g1)
        return np.array([lp_0g0, lp_0g1, lp_1g0, lp_1g1])  # 4 by nv by m

    def calculate_alpha(self, X, p_y_given_x, theta, log_p_y, tcs):
        """A rule for non-tree CorEx structure."""
        # TODO: Could make it sparse also? Well, maybe not... at the beginning it's quite non-sparse
        mis = self.calculate_mis(theta, log_p_y)
        if self.n_hidden == 1:
            alphaopt = np.ones((1, self.n_visible))
        elif self.tree:
            # sa = np.sum(self.alpha, axis=0)
            tc_oom = 1. / self.n_samples
            sa = np.sum(self.alpha[tcs > tc_oom], axis=0)
            self.t = np.where(sa > 1.1, 1.3 * self.t, self.t)
            # tc_oom = np.median(self.h_x)  # \propto TC of a small group of corr. variables w/median entropy...
            # t = 20 + (20 * np.abs(tcs) / tc_oom).reshape((self.n_hidden, 1))  # worked well in many tests
            t = (1 + self.t * np.abs(tcs).reshape((self.n_hidden, 1)))
            maxmis = np.max(mis, axis=0)
            for i in np.where((mis == maxmis).sum(axis=0))[0]:  # Break ties for the largest MI
                mis[:, i] += 1e-10 * np.random.random(self.n_hidden)
                maxmis[i] = np.max(mis[:, i])
            with np.errstate(under='ignore'):
                alphaopt = np.exp(t * (mis - maxmis) / self.h_x)
        else:
            # TODO: Can we make a fast non-tree version of update in the AISTATS paper?
            alphaopt = np.zeros((self.n_hidden, self.n_visible))
            top_ys = np.argsort(-mis, axis=0)[:self.tree]
            raise NotImplementedError
        self.mis = mis  # So we don't have to recalculate it when used later
        return alphaopt

    def calculate_latent(self, X, theta):
        """"Calculate the probability distribution for hidden factors for each sample."""
        ns, nv = X.shape
        log_pygx_unnorm = np.empty((2, ns, self.n_hidden))
        c0 = np.einsum('ji,ij->j', self.alpha, theta[0] - self.lp0)
        c1 = np.einsum('ji,ij->j', self.alpha, theta[1] - self.lp0)  # length n_hidden
        info0 = np.einsum('ji,ij->ij', self.alpha, theta[2] - theta[0] + self.px_frac)
        info1 = np.einsum('ji,ij->ij', self.alpha, theta[3] - theta[1] + self.px_frac)
        log_pygx_unnorm[1] = self.log_p_y + c1 + X.dot(info1)
        log_pygx_unnorm[0] = log_1mp(self.log_p_y) + c0 + X.dot(info0)
        return self.normalize_latent(log_pygx_unnorm)

    def normalize_latent(self, log_pygx_unnorm):
        """Normalize the latent variable distribution
        For each sample in the training set, we estimate a probability distribution
        over y_j, each hidden factor. Here we normalize it. (Eq. 7 in paper.)
        This normalization factor is used for estimating TC.
        Parameters
        ----------
        Unnormalized distribution of hidden factors for each training sample.
        Returns
        -------
        p_y_given_x : 3D array, shape (n_hidden, n_samples)
            p(y_j|x^l), the probability distribution over all hidden factors,
            for data samples l = 1...n_samples
        log_z : 2D array, shape (n_hidden, n_samples)
            Point-wise estimate of total correlation explained by each Y_j for each sample,
            used to estimate overall total correlation.
        """
        with np.errstate(under='ignore'):
            log_z = logsumexp(log_pygx_unnorm, axis=0)  # Essential to maintain precision.
            log_pygx = log_pygx_unnorm[1] - log_z
            p_norm = np.exp(log_pygx)
        return p_norm.clip(1e-6, 1 - 1e-6), log_pygx, log_z  # ns by m

    def update_tc(self, log_z):
        self.tcs = np.mean(log_z, axis=0)
        self.tc_history.append(np.sum(self.tcs))

    def print_verbose(self):
        if self.verbose:
            print(self.tcs)
        if self.verbose > 1:
            print(self.alpha[:, :, 0])
            print(self.theta)

    def convergence(self):
        if len(self.tc_history) > 10:
            dist = -np.mean(self.tc_history[-10:-5]) + np.mean(self.tc_history[-5:])
            return np.abs(dist) < self.eps  # Check for convergence.
        else:
            return False

    def __getstate__(self):
        # In principle, if there were variables that are themselves classes... we have to handle it to pickle correctly
        # But I think I programmed around all that.
        self_dict = self.__dict__.copy()
        return self_dict

    def save(self, filename):
        """ Pickle a class instance. E.g., corex.save('saved.dat') """
        # Avoid saving words with object.
        #TODO: figure out why Unicode sometimes causes an issue with loading after pickling
        if self.words is not None:
            temp_words = self.words
            self.words = None
        else:
            temp_words = None
        # Save CorEx object
        import pickle
        if path.dirname(filename) and not path.exists(path.dirname(filename)):
            makedirs(path.dirname(filename))
        pickle.dump(self, open(filename, 'wb'), protocol=-1)
        # Restore words to CorEx object
        self.words = temp_words

    def save_joblib(self, filename):
        """ Serialize a class instance with joblib - better for larger models. E.g., corex.save('saved.dat') """
        # Avoid saving words with object.
        if self.words is not None:
            temp_words = self.words
            self.words = None
        else:
            temp_words = None
        # Save CorEx object
        if path.dirname(filename) and not path.exists(path.dirname(filename)):
            makedirs(path.dirname(filename))
        joblib.dump(self, filename)
        # Restore words to CorEx object
        self.words = temp_words

    def sort_and_output(self, X):
        order = np.argsort(self.tcs)[::-1]  # Order components from strongest TC to weakest
        self.tcs = self.tcs[order]  # TC for each component
        self.alpha = self.alpha[order]  # Connections between X_i and Y_j
        self.log_p_y = self.log_p_y[order]  # Parameters defining the representation
        self.theta = self.theta[:, :, order]  # Parameters defining the representation

    def calculate_mis(self, theta, log_p_y):
        """Return MI in nats, size n_hidden by n_variables"""
        p_y = np.exp(log_p_y).reshape((-1, 1))  # size n_hidden, 1
        mis = self.h_x - p_y * binary_entropy(np.exp(theta[3]).T) - (1 - p_y) * binary_entropy(np.exp(theta[2]).T)
        return (mis - 1. / (2. * self.n_samples)).clip(0.)  # P-T bias correction

    def get_topics(self, n_words=10, topic=None, print_words=True):
        """
        Return list of lists of tuples. Each list consists of the top words for a topic
        and each tuple is a pair (word, mutual information). If 'words' was not provided
        to CorEx, then 'word' will be an integer column index of X
        topic_n : integer specifying which topic to get (0-indexed)
        print_words : boolean, get_topics will attempt to print topics using
                      provided column labels (through 'words') if possible. Otherwise,
                      topics will be consist of column indices of X
        """
        # Determine which topics to iterate over
        if topic is not None:
            topic_ns = [topic]
        else:
            topic_ns = list(range(self.labels.shape[1]))
        # Determine whether to return column word labels or indices
        if self.words is None:
            print_words = False
            print("NOTE: 'words' not provided to CorEx. Returning topics as lists of column indices")
        elif len(self.words) != self.alpha.shape[1]:
            print_words = False
            print('WARNING: number of column labels != number of columns of X. Cannot reliably add labels to topics. Check len(words) and X.shape[1]. Use .set_words() to fix')

        topics = [] # TODO: make this faster, it's slower than it should be
        for n in topic_ns:
            # Get indices of which words belong to the topic
            inds = np.where(self.alpha[n] >= 1.)[0]
            # Sort topic words according to mutual information
            inds = inds[np.argsort(-self.alpha[n,inds] * self.mis[n,inds])]
            # Create topic to return
            if print_words is True:
                topic = [(self.col_index2word[ind], self.sign[n,ind]*self.mis[n,ind]) for ind in inds[:n_words]]
            else:
                topic = [(ind, self.sign[n,ind]*self.mis[n,ind]) for ind in inds[:n_words]]
            # Add topic to list of topics if returning all topics. Otherwise, return topic
            if len(topic_ns) != 1:
                topics.append(topic)
            else:
                return topic

        return topics

    def get_top_docs(self, n_docs=10, topic=None, sort_by='log_prob', print_docs=True):
        """
        Return list of lists of tuples. Each list consists of the top docs for a topic
        and each tuple is a pair (doc, pointwise TC or probability). If 'docs' was not
        provided to CorEx, then each doc will be an integer row index of X
        topic_n : integer specifying which topic to get (0-indexed)
        sort_by: 'log_prob' or 'tc', use either 'log_p_y_given_x' or 'log_z' respectively
                 to return top docs per each topic
        print_docs : boolean, get_top_docs will attempt to print topics using
                     provided row labels (through 'docs') if possible. Otherwise,
                     top docs will be consist of row indices of X
        """
        # Determine which topics to iterate over
        if topic is not None:
            topic_ns = [topic]
        else:
            topic_ns = list(range(self.labels.shape[1]))
        # Determine whether to return row doc labels or indices
        if self.docs is None:
            print_docs = False
            print("NOTE: 'docs' not provided to CorEx. Returning top docs as lists of row indices")
        elif len(self.docs) != self.labels.shape[0]:
            print_words = False
            print('WARNING: number of row labels != number of rows of X. Cannot reliably add labels. Check len(docs) and X.shape[0]. Use .set_docs() to fix')
        # Get appropriate matrix to sort
        if sort_by == 'log_prob':
            doc_values = self.log_p_y_given_x
        elif sort_by == 'tc':
            print('WARNING: sorting by logz not well tested')
            doc_values = self.log_z
        else:
            print("Invalid 'sort_by' parameter, must be 'prob' or 'tc'")
            return
        # Get top docs for each topic
        doc_inds = np.argsort(-doc_values, axis=0)
        top_docs = [] # TODO: make this faster, it's slower than it should be
        for n in topic_ns:
            if print_docs is True:
                topic_docs = [(self.row_index2doc[ind], doc_values[ind,n]) for ind in doc_inds[:n_docs,n]]
            else:
                topic_docs = [(ind, doc_values[ind,n]) for ind in doc_inds[:n_docs,n]]
            # Add docs to list of top docs per topic if returning all topics. Otherwise, return
            if len(topic_ns) != 1:
                top_docs.append(topic_docs)
            else:
                return topic_docs

        return top_docs

    def set_words(self, words):
        self.words = words
        if words is not None:
            if len(words) != self.alpha.shape[1]:
                print('WARNING: number of column labels != number of columns of X. Check len(words) and .alpha.shape[1]')
            col_index2word = {index:word for index,word in enumerate(words)}
            word2col_index = {word:index for index,word in enumerate(words)}
            self.col_index2word = col_index2word
            self.word2col_index = word2col_index

    def set_docs(self, docs):
        self.docs = docs
        if docs is not None:
            if len(docs) != self.labels.shape[0]:
                print('WARNING: number of row labels != number of rows of X. Check len(docs) and .labels.shape[0]')
            row_index2doc = {index:doc for index,doc in enumerate(docs)}
            self.row_index2doc = row_index2doc


def log_1mp(x):
    return np.log1p(-np.exp(x))


def binary_entropy(p):
    return np.where(p > 0, - p * np.log2(p) - (1 - p) * np.log2(1 - p), 0)


def flatten(a):
    b = []
    for ai in a:
        if type(ai) is list:
            b += ai
        else:
            b.append(ai)
    return b


def load(filename):
    """ Unpickle class instance. """
    import pickle
    return pickle.load(open(filename, 'rb'))


def load_joblib(filename):
    """ Load class instance with joblib. """
    return joblib.load(filename)

In [83]:
import scipy.sparse as ss
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import string
import re
from sklearn.feature_extraction.text import CountVectorizer

In [84]:
# read input data which is in .txt file line by line, split by tab(\t) to a list
list_data=[]
with open('C:\\Users\\Gaurav\\Desktop\\ISI forms\\Project\\il6_original.txt',encoding='utf8',errors='ignore') as fp:
    for line in fp:
        list_data.append(line.split('\t'))

In [85]:
# name the columns of datframe
raw_data = pd.DataFrame(list_data,columns=['doc_id','text_data','class_type','additional']) 
#drop if any additional columns gets created as part of reading process
raw_data=raw_data.drop('additional',axis=1)
# strip columns for leading and trailing white spaces
raw_data['doc_id']=raw_data.doc_id.str.strip()
raw_data['text_data']=raw_data.text_data.str.strip()
raw_data['class_type']=raw_data['class_type'].str.strip()
# n docs x m attributes
raw_data.shape

(7016, 3)

In [86]:
# set doc_id as index
raw_data= raw_data.set_index('doc_id')
# change "class_type" column to "categorical" datatype
raw_data['class_type'] = raw_data['class_type'].astype('category')
raw_data['class_type'].unique()

[eval_incident, unk, nondomain, indomain]
Categories (4, object): [eval_incident, unk, nondomain, indomain]

In [87]:
#eval_incident - This should be considered as part of “indomain”
raw_data.loc[raw_data['class_type']=="eval_incident", 'class_type'] = "indomain"

In [88]:
# http links removal on 'text_data' column
# regex : ((http|https)://t.co/[a-zA-Z0-9]+)

raw_data['text_data'] = raw_data['text_data'].apply(lambda x: re.sub('((http|https)://t.co/[a-zA-Z0-9]+)','',x))

# RT (Retweet) keyword removal

raw_data['text_data'] = raw_data['text_data'].apply(lambda x: re.sub('RT','',x))

#punctuation removal on 'text_data' column
#print(string.punctuation)
punct='!"$%&()*+,-./:;<=>?[\]^_`{|}~'+"'"
regex = re.compile('[%s]' % re.escape(punct))
raw_data['text_data'] = raw_data['text_data'].apply(lambda x: regex.sub('', x))
raw_data['text_data']=raw_data.text_data.str.strip()

# remove @names mentioned as part of tweets
raw_data['text_data'] = raw_data['text_data'].apply(lambda x: re.sub('\@[a-zA-Z0-9]+','',x))

# Remove emoji's from the documents
raw_data['text_data'] = raw_data['text_data'].apply(lambda x: re.sub('["\U0001F600-\U0001F64F" "\U0001F300-\U0001F5FF" "\U0001F680-\U0001F6FF"  "\U0001F1E0-\U0001F1FF"]+',' ',x))

# Date Removal/ number removal from the documents
raw_data['text_data'] = raw_data['text_data'].apply(lambda x: re.sub('[\d]+','',x))
# strip whitespaces again 
raw_data['text_data']=raw_data.text_data.str.strip()
raw_data['class_type']=raw_data['class_type'].str.strip()

# Convert the letters to lower case
raw_data['text_data'] = raw_data['text_data'].apply(lambda x: x.lower())

# calculate length again
raw_data['length'] = raw_data['text_data'].apply(lambda x: len(x.split()))
# sort by file length
raw_data=raw_data.sort_values(by='length', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [89]:
# remove the documents with no words after pre-processing
raw_data = raw_data.loc[raw_data.length>0]
raw_data.head(2)

,text_data,class_type,length
doc_id,,,
IL6_SN_000370_20170330_H0T003OA1,こっちがウサギタロッ,unk,1
IL6_SN_000370_20170329_H0T003FRR,ゴスロリバイブルの方はmana様が載ってるからまだ買ってたんだけど、ケラまでは買ってなかった...,unk,1


In [90]:
#Removing documents with less than 5 words
raw_data = raw_data.loc[raw_data.length>5]
raw_data.shape

(5871, 3)

In [91]:
# Document count across 3 class labels (unk,nondomain,indomain,eval_incident)
print(raw_data.class_type.value_counts())
plt.hist('class_type',data=raw_data)
plt.show()

unk          3901
indomain     1806
nondomain     164
Name: class_type, dtype: int64


TypeError: len() of unsized object

In [95]:
# write the formatted data to a new text file for future reference

reset_data=raw_data.reset_index()
reset_data.to_csv('C:\\Users\\Gaurav\\Desktop\\ISI forms\\Project\\Oromo.txt', header=True, index=False, sep='\t', mode='w',encoding='utf-8')


In [96]:
# basic stats to understand document length across 4 class labels
raw_data.groupby('class_type').describe()

length                                                       
             count        mean         std  min   25%   50%   75%     max
class_type                                                               
indomain    1806.0   35.859911  107.356458  6.0  10.0  12.0  14.0   981.0
nondomain    164.0   42.073171  112.330503  6.0  11.0  13.0  14.0   684.0
unk         3901.0  125.700333  344.687689  6.0   9.0  14.0  27.0  6062.0

In [97]:
# Document Term matrix (binary matrix) with max_df =0.995, min_df =0.001
vectorizer = CountVectorizer(encoding='utf-8',stop_words=None,max_df =0.995, min_df =0.001, binary = True,lowercase=True)
doc_word_mat = vectorizer.fit_transform(reset_data.text_data)
doc_word_mat = ss.csr_matrix(doc_word_mat)
doc_word_mat.shape # n_docs x m_words

(5871, 8360)

In [98]:
# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words = list(np.asarray(vectorizer.get_feature_names()))

In [99]:
# Train the CorEx topic model with 25 topics
topic_model = Corex(n_hidden=25, words=words, max_iter=4000, verbose=False, seed=3192)
topic_model.fit(doc_word_mat, words=words)

In [100]:
# Print all topics from the CorEx topic model
topics = topic_model.get_topics(n_words=10)
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: qabsoo,ummata,of,ilmaan,oromoon,diina,bilisummaa,seenaa,oromoof,saba
1: akka,kana,jiru,isaa,irraa,kun,keessa,isaanii,itti,jira
2: garuu,nama,miti,inni,jedhee,harka,jedhanii,ittiin,biraa,jecha
3: abbaa,osoo,gadi,qofa,ni,hundi,hunda,ifatti,ammas,garaa
4: warra,warri,nutti,beeku,koo,jirti,ani,natti,jechaa,kiyya
5: hin,oromoo,wal,keenya,karaa,dha,haa,nu,amma,jiruu
6: wayyaanee,humna,wayyaaneen,waraana,irree,tarkaanfii,humni,qeerroon,mooraa,qeerroo
7: qabu,biyya,isa,kanaaf,taane,ofii,siyaasaa,biyyaa,yaada,aadaa
8: rakkoo,alaa,taʼeef,dandaʼu,biyyattii,tokkummaa,dhabuu,fakkaatu,hawaasaa,akkanaa
9: bifa,jalatti,ajjeechaa,qabnu,qabaachuu,bittaa,baasuu,umrii,dimokraasii,bilisa
10: mirga,taʼee,aangoo,biyyummaa,mara,ragaa,jirru,harkaa,dhaaba,lafaa
11: irra,oromiyaa,turan,kanaa,dhimma,gama,namoota,guutuu,isaaf,as
12: dura,sadarkaa,waggaa,waggoota,turee,barnootaa,dhaabbachuun,dhaabbilee,tajaajila,argatan
13: fi,irratti,waliin,bara,haala,addaa,gaaffii,akkasumas,deebii,afaan
14: sirna,barbaachisu,f

In [101]:
# Anchor words
#["chocho'a", "lafaa","socho'a","lafa"], 
#socho始aa socho始an socho始uun  socho始uu  
anchor_words = [["hoongee","hongee"],["galaana","galaanaa"],["balaa"]]
anchored_topic_model = Corex(n_hidden=25, max_iter=4000,seed=3192)
anchored_topic_model.fit(doc_word_mat, words=words, anchors=anchor_words, anchor_strength=6);

In [102]:
for n in range(len(anchor_words)):
    topic_words,_ = zip(*anchored_topic_model.get_topics(topic=n))
    print('{}: '.format(n) + ','.join(topic_words))

0: aadaa,isaaniif,jirru,hongee,qofaa,dhuma,caalaatti,adeemsa,qabatanii,hoongee
1: malee,of,oromoon,seenaa,gadi,harka,hunda,ni,saba,hundi
2: balaa,fakkaatu,isaatiin,cunqursaa,dabree,mootummoota,faallaa,habashaa,odoo,qbo


In [103]:
#softmax binary classification of which documents belong to the topics of anchored words
#(in this case, we have words seeded for 3 topics, hence checking the classification for those topics) 
topic_classification=anchored_topic_model.labels[:,0:3] # n_docs x k_topics

In [104]:
# Storing the classification results for the first 3 topics (seeded)
result=pd.DataFrame(topic_classification,columns=["topic1","topic2","topic3"])

In [105]:
raw_data.reset_index(inplace=True)
raw_data.head(2)

,doc_id,text_data,class_type,length
0,IL6_SN_000370_20170331_H0T003SSC,ai mana que dor de cabeça,unk,6
1,IL6_SN_000370_20160822_H0T0060G0,ilma ummatni oromoo itti boonuu qabu,indomain,6


In [106]:
result = pd.concat( [reset_data.iloc[:,0:3], result], axis=1)
result.head(2)

,doc_id,text_data,class_type,topic1,topic2,topic3
0,IL6_SN_000370_20170331_H0T003SSC,ai mana que dor de cabeça,unk,False,False,False
1,IL6_SN_000370_20160822_H0T0060G0,ilma ummatni oromoo itti boonuu qabu,indomain,False,False,False


In [107]:
# assigning class lables based on the binary classification result
result.loc[(result['topic1']==True) | (result['topic2']==True) |  (result['topic3']==True) , 'predicted_class_label'] = "indomain"
result.loc[(result['topic1']==False) & (result['topic2']==False) & (result['topic3']==False)  , 'predicted_class_label'] = "nondomain"

In [108]:
# filter unknown class
print("shape before filtering unknown docs",result.shape )
result = result.loc[result.class_type !="unk"]
print("shape after filtering unknown docs",result.shape )

shape before filtering unknown docs (5871, 7)
shape after filtering unknown docs (1970, 7)


In [109]:
from sklearn.metrics import confusion_matrix
cm=pd.DataFrame(confusion_matrix(result.class_type, result.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])
print(cm,"\n")

precision = cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

recall =  cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++          121         1685
Actual--           16          148 

precision 0.883211678832 

recall 0.0669988925803 

F1_score 0.124549665466 



In [110]:
# more Anchor words from chineaseroom translator
#["chocho'a", "lafaa","socho'a","lafa"], 
#socho始aa socho始an socho始uun  socho始uu  

anchor_words = [["hoongee","hongee","oolaa"],["galaana","galaanaa","guutuu","lolaa"],
                ["balaa","dhibee","irraa"],
                ["sochii","lafaa"]]

anchored_topic_model = Corex(n_hidden=25, max_iter=4000,seed=3192)
anchored_topic_model.fit(doc_word_mat, words=words, anchors=anchor_words, anchor_strength=6);

In [111]:
for n in range(len(anchor_words)):
    topic_words,_ = zip(*anchored_topic_model.get_topics(topic=n))
    print('{}: '.format(n) + ','.join(topic_words))

0: qofa,waggoota,meeshaa,turee,kanas,isaatti,hongee,ammo,jennee,kkf
1: guutuu,lolaa,galaana,galaanaa,dhimmi,baasuu,taʼanii,biraatiin,dimokraasii,dhaggeeffadhaa
2: irraa,balaa,dhibee,oggaa,kenname,galmee,saʼa,karaan,labsa,jijjiirraa
3: sochii,lafaa,barbaachisu,gaggeessaa,dhabuun,barbaada,adeemsa,olaanaa,lammiilee,warraaqsa


In [112]:
#softmax binary classification of which documents belong to the topics of anchored words
#(in this case, we have words seeded for 4 topics, hence checking the classification for those topics) 
topic_classification=anchored_topic_model.labels[:,0:4] # n_docs x k_topics

In [113]:
# Storing the classification results for the first 4 topics (seeded)
result=pd.DataFrame(topic_classification,columns=["topic1","topic2","topic3","topic4"])

In [114]:
result = pd.concat( [reset_data.iloc[:,0:3], result], axis=1)
result.head(2)

,doc_id,text_data,class_type,topic1,topic2,topic3,topic4
0,IL6_SN_000370_20170331_H0T003SSC,ai mana que dor de cabeça,unk,False,False,False,False
1,IL6_SN_000370_20160822_H0T0060G0,ilma ummatni oromoo itti boonuu qabu,indomain,False,False,False,False


In [115]:
# assigning class lables based on the binary classification result
result.loc[(result['topic1']==True) | (result['topic2']==True) |  (result['topic3']==True) |(result['topic4']==True) , 'predicted_class_label'] = "indomain"
result.loc[(result['topic1']==False) & (result['topic2']==False) & (result['topic3']==False) & (result['topic4']==False)  , 'predicted_class_label'] = "nondomain"

In [116]:
# filter unknown class
print("shape before filtering unknown docs",result.shape )
result = result.loc[result.class_type !="unk"]
print("shape after filtering unknown docs",result.shape )

shape before filtering unknown docs (5871, 8)
shape after filtering unknown docs (1970, 8)


In [117]:
from sklearn.metrics import confusion_matrix
cm=pd.DataFrame(confusion_matrix(result.class_type, result.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])

print("After adding more words from chinease room translator \n" )
print(cm,"\n")


precision = cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

recall =  cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

After adding more words from chinease room translator 

          Predicted++  Predicted--
Actual++          250         1556
Actual--           27          137 

precision 0.902527075812 

recall 0.138427464009 

F1_score 0.240038406145 



In [118]:
# metrics @ top 20 based anchor word count

top_docs = result
top_docs['count'] = top_docs['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'hoongee|hongee|oolaa|galaana|galaanaa|guutuu|lolaa|balaa|dhibee|irraa|sochii|lafaa').sum())
top_docs.sort_values(by=['count'],ascending=False,inplace=True)
top15 = top_docs.iloc[0:20,:]
cm=pd.DataFrame(confusion_matrix(top15.class_type, top15.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])
print(cm,"\n")
# precision
precision = cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

#recall
recall =  cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

# F1-Score
F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++           19            0
Actual--            1            0 

precision 0.95 

recall 1.0 

F1_score 0.974358974359 



In [119]:
# metrics @ top 50 based anchor word count

top_docs = result
top_docs['count'] = top_docs['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'hoongee|hongee|oolaa|galaana|galaanaa|guutuu|lolaa|balaa|dhibee|irraa|sochii|lafaa').sum())
top_docs.sort_values(by=['count'],ascending=False,inplace=True)
top15 = top_docs.iloc[0:50,:]
cm=pd.DataFrame(confusion_matrix(top15.class_type, top15.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])
print(cm,"\n")
# precision
precision = cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

#recall
recall =  cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

# F1-Score
F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++           46            3
Actual--            1            0 

precision 0.978723404255 

recall 0.938775510204 

F1_score 0.958333333333 



In [120]:
# Top 10 docs for topic 1


topic1_docs_id = result.loc[(result.class_type=="indomain")&(result.topic1==True),"doc_id"]

topic1_docs_id.shape

topic1_docs =raw_data.loc[raw_data.doc_id.isin(topic1_docs_id),["doc_id","text_data"]]
topic1_docs= topic1_docs.set_index('doc_id')

#anchor_words = [["hoongee","hongee","oolaa"],["galaana","galaanaa","guutuu","lolaa"],["balaa","dhibee","irraa"],["sochii","lafaa"]]

topic1_docs['count'] = topic1_docs['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'hoongee|hongee|oolaa').sum())

topic1_docs.sort_values(by=['count'],ascending=False,inplace=True)

topic1_docs.iloc[0:10,:]

,text_data,count
doc_id,,
IL6_NW_020423_20151002_H0040LX9Z,itoophiyaa keessatti laakkofsi namoota hongeef...,7
IL6_WL_020537_20161206_H0040MR72,balaa hoongee naannolee shanitti uumameef deeg...,7
IL6_NW_020532_20170426_H0040MPVA,aanaalee godina guraagee balaan hoongee mudate...,5
IL6_NW_020505_20150902_H0040MO8I,ijoo dubbii abo ummatoota beela irraa baraaruu...,4
IL6_NW_020423_20160205_H0040LX8T,ummata beelaye duʼa jalaa baraaruuf gargaarsa ...,4
IL6_WL_020535_20151124_H0040MR6K,diiniyaas alamuu laliisaa itiyoopiyaan maalif ...,4
IL6_NW_020412_20150929_H0040MR7V,hoongee fi gogiinsa oromiyaa keessaa lammiilee...,3
IL6_NW_020533_20170318_H0040MO5O,sababa walitti buʼiinsa daangaa oromiyaa fi su...,3
IL6_NW_020533_20170120_H0040MO5P,daangaa naannoo oromiyaa fi sumaalee irratti w...,2


In [121]:
# Top 10 docs for topic 2


topic2_docs_id = result.loc[(result.class_type=="indomain")&(result.topic2==True),"doc_id"]

topic2_docs_id.shape


topic2_docs =raw_data.loc[raw_data.doc_id.isin(topic2_docs_id),["doc_id","text_data"]]
topic2_docs= topic2_docs.set_index('doc_id')

#anchor_words = [["hoongee","hongee","oolaa"],["galaana","galaanaa","guutuu","lolaa"],["balaa","dhibee","irraa"],["sochii","lafaa"]]

topic2_docs['count'] = topic2_docs['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'galaana|galaanaa|guutuu|lolaa').sum())

topic2_docs.sort_values(by=['count'],ascending=False,inplace=True)

topic2_docs.iloc[0:10,:]

,text_data,count
doc_id,,
IL6_WL_020415_20160225_H0040LT3D,warraaqsi bilisummaa oromoo finiinaajiru sochi...,16
IL6_NW_020445_20160426_H0040M4CT,balaa lolaa dirree dawaatti qaqqabeen namoonni...,8
IL6_NW_020423_20160419_H0040LX80,biyyatti ajjeefamuu fi biyyoota ollaattii abdi...,7
IL6_NW_020532_20160105_H0040MPV6,koomishinichi karoora balaa lolaa ittisuu fi l...,7
IL6_NW_020532_20170426_H0040MPV3,rooba tibbanaan oromiyaatti namoonni kuma taʼ...,6
IL6_WL_020423_20150814_H0040MR7C,haalli qilleensa el nino cimaan uumamuuf deema...,6
IL6_NW_020445_20160507_H0040M4CF,ministeerichi balaa lolaa mudatu hirʼisuuf hoj...,6
IL6_NW_020117_20160705_H0040LMLC,bokkaan kalee barraaqa adaamaatti roobe waan h...,5
IL6_NW_020427_20170210_H0040LYS7,mudannoon weerara maqaa liyyuu polisiin murnii...,5


In [122]:
# Top 10 docs for topic 3


topic3_docs_id = result.loc[(result.class_type=="indomain")&(result.topic3==True),"doc_id"]

topic3_docs_id.shape


topic3_docs =raw_data.loc[raw_data.doc_id.isin(topic3_docs_id),["doc_id","text_data"]]
topic3_docs= topic3_docs.set_index('doc_id')

#anchor_words = ["balaa","dhibee","irraa"],["sochii","lafaa"]]

topic3_docs['count'] = topic3_docs['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'balaa|dhibee|irraa').sum())

topic3_docs.sort_values(by=['count'],ascending=False,inplace=True)
topic3_docs.iloc[0:10,:]

,text_data,count
doc_id,,
IL6_WL_020539_20160403_H0040MR7Q,bishaantu rasaasa taʼe yooseef hambaa tolaa bi...,22
IL6_WL_020405_20170307_H0040LOE5,sabummaa mormaa sabummaa deggeruu sbo bitootes...,18
IL6_WL_020415_20160225_H0040LT3D,warraaqsi bilisummaa oromoo finiinaajiru sochi...,16
IL6_WL_020405_20170307_H0040LOFT,shira lukkeelee fi gooftoota isaani gaomee dhu...,16
IL6_NW_020532_20160105_H0040MPV6,koomishinichi karoora balaa lolaa ittisuu fi l...,13
IL6_NW_020411_20170318_H0040LSJJ,sirni tpleprdf wayta ummata wayyaba gara lammi...,12
IL6_WL_020412_20170427_H0040MR6F,ibsa ijjannoo barattoota oromoo yuunibarsiitii...,11
IL6_NW_020117_20160420_H0040LMNX,balaa doonii godaantota galaafaterraa namni l...,11
IL6_NW_020423_20160419_H0040LX80,biyyatti ajjeefamuu fi biyyoota ollaattii abdi...,11


In [123]:
# Top 10 docs for topic 4


topic4_docs_id = result.loc[(result.class_type=="indomain")&(result.topic4==True),"doc_id"]

topic4_docs_id.shape


topic4_docs =raw_data.loc[raw_data.doc_id.isin(topic4_docs_id),["doc_id","text_data"]]
topic4_docs= topic4_docs.set_index('doc_id')

#anchor_words = ["balaa","dhibee","irraa"],["sochii","lafaa"]]

topic4_docs['count'] = topic4_docs['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'sochii|lafaa').sum())

topic4_docs.sort_values(by=['count'],ascending=False,inplace=True)
topic4_docs.iloc[0:10,:]

,text_data,count
doc_id,,
IL6_WL_020415_20160225_H0040LT3D,warraaqsi bilisummaa oromoo finiinaajiru sochi...,7
IL6_WL_020405_20170307_H0040LOET,qeerroo diddaan barattoota oromoo bifa qindaay...,6
IL6_WL_020525_20160831_H0040MNBX,#oromoprotests #dhaamsa qeerroo bilisummaa oro...,5
IL6_WL_020405_20170307_H0040LOD7,karoora mootummaan wayyaanee ergamtuu isaa opd...,5
IL6_WL_020520_20160716_H0040MMVK,injifannoo uummanni oromoo baatii saddet darba...,4
IL6_WL_020405_20170307_H0040LOFT,shira lukkeelee fi gooftoota isaani gaomee dhu...,4
IL6_WL_020405_20170307_H0040LOSG,wayyaaneen shira marsaa ffaa magaalaa amboo ir...,4
IL6_NW_020423_20150514_H0040LXAY,sochiin dachii oromiyaa eprdfopdo irratti mana...,3
IL6_NW_020427_20150806_H0040LYRC,uummatni hidhaa fi reebichaan saree iyyuu ni x...,3


In [124]:
## ranking

temp = result.loc[result.class_type =="indomain"]
result_nondomain = result.loc[result.class_type =="nondomain"]
result_indomain=temp.sample(n=result_nondomain.shape[0],  replace=False, random_state=3192, axis=None)
balanced_result= pd.concat( [result_indomain, result_nondomain], axis=0,ignore_index=True)

In [125]:
# metrics @ top 20 based anchor word count

top_docs = balanced_result
top_docs['count'] = top_docs['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'hoongee|hongee|oolaa|galaana|galaanaa|guutuu|lolaa|balaa|dhibee|irraa|sochii|lafaa').sum())
top_docs.sort_values(by=['count'],ascending=False,inplace=True)
top20 = top_docs.iloc[0:20,:]
cm=pd.DataFrame(confusion_matrix(top20.class_type, top20.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])
print(cm,"\n")
# precision
precision = cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

#recall
recall =  cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

# F1-Score
F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++            7            1
Actual--           12            0 

precision 0.368421052632 

recall 0.875 

F1_score 0.518518518519 



In [126]:
##tf 

# metrics @ top 20 based anchor word count

top_docs = balanced_result

top_docs['tf'] = top_docs['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'hoongee|hongee|oolaa|galaana|galaanaa|guutuu|lolaa|balaa|dhibee|irraa|sochii|lafaa').sum()/len(x.split()))

top_docs.sort_values(by=['tf'],ascending=False,inplace=True)
top20 = top_docs.iloc[0:20,:]
cm=pd.DataFrame(confusion_matrix(top20.class_type, top20.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])
print(cm,"\n")
# precision
precision = cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

#recall
recall =  cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

# F1-Score
F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++            6            5
Actual--            8            1 

precision 0.428571428571 

recall 0.545454545455 

F1_score 0.48 



In [127]:
##tf sum

# metrics @ top 20 based anchor word count

top_docs = balanced_result

top_docs['tf'] = top_docs['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'hoongee|hongee|oolaa|galaana|galaanaa|guutuu|lolaa|balaa|dhibee|irraa|sochii|lafaa').sum()/len(x.split()))

top_docs.sort_values(by=['tf'],ascending=False,inplace=True)
top50 = top_docs.iloc[0:50,:]
cm=pd.DataFrame(confusion_matrix(top50.class_type, top50.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])
print(cm,"\n")
# precision
precision = cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

#recall
recall =  cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

# F1-Score
F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++           16            9
Actual--           21            4 

precision 0.432432432432 

recall 0.64 

F1_score 0.516129032258 



In [128]:
# metrics @ top 50 based anchor word count

top_docs = balanced_result
top_docs['count'] = top_docs['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'hoongee|hongee|oolaa|galaana|galaanaa|guutuu|lolaa|balaa|dhibee|irraa|sochii|lafaa').sum())
top_docs.sort_values(by=['count'],ascending=False,inplace=True)
top50 = top_docs.iloc[0:50,:]
cm=pd.DataFrame(confusion_matrix(top50.class_type, top50.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])
print(cm,"\n")
# precision
precision = cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

#recall
recall =  cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

# F1-Score
F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++           15            8
Actual--           24            3 

precision 0.384615384615 

recall 0.652173913043 

F1_score 0.483870967742 



In [129]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(encoding='utf-8',stop_words=None, max_df =0.997, min_df =1,binary =False,
                             lowercase=True, use_idf=True,smooth_idf=True)

tf_idf_mat = vectorizer.fit_transform(balanced_result.text_data)
tf_idf_mat.shape # n_docs x m_words

(328, 4786)

In [130]:
dense_matrix=tf_idf_mat.todense()
keyword_mat=dense_matrix[:,[2184,1680,1681,1911,2950,433,2353,3966,2894]]

In [131]:
pd2=pd.DataFrame(keyword_mat)

In [132]:
# calculate word count (length) of the document
pd2['sum'] = pd2.apply(sum, axis=1)

# sort by highest td-idfscores
pd2=pd2.sort_values(by='sum', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')


In [133]:
#top 20
top20_index=pd2[0:20].index.tolist()
top20_index


[144,
 224,
 320,
 117,
 195,
 265,
 31,
 11,
 27,
 4,
 223,
 222,
 221,
 220,
 219,
 0,
 218,
 217,
 215,
 214]

In [134]:
top20 = balanced_result.iloc[top20_index]
type(top20)
#top20.shape

top20_df=pd.DataFrame(confusion_matrix(top20.class_type, top20.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])

print(top20_df,"\n")
# precision
precision = top20_df.loc["Actual++","Predicted++"]/(top20_df.loc["Actual++","Predicted++"] + top20_df.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

#recall
recall =  top20_df.loc["Actual++","Predicted++"]/(top20_df.loc["Actual++","Predicted++"] + top20_df.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

# F1-Score
F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")



          Predicted++  Predicted--
Actual++            0            2
Actual--            5           13 

precision 0.0 

recall 0.0 

F1_score nan 



In [135]:
#top 20
top50_index=pd2[0:50].index.tolist()
top50_index
top50 = balanced_result.iloc[top50_index]
type(top20)
#top20.shape

top50_df=pd.DataFrame(confusion_matrix(top50.class_type, top50.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])

print(top50_df,"\n")
# precision
precision = top50_df.loc["Actual++","Predicted++"]/(top50_df.loc["Actual++","Predicted++"] + top50_df.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

#recall
recall =  top50_df.loc["Actual++","Predicted++"]/(top50_df.loc["Actual++","Predicted++"] + top50_df.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

# F1-Score
F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")



          Predicted++  Predicted--
Actual++            0            4
Actual--            6           40 

precision 0.0 

recall 0.0 

F1_score nan 



In [136]:
cd=pd2.iloc[:,0:9].copy()

In [137]:
# calculate word count (length) of the document
cd['product'] = cd.prod(axis=1)
cd.head(2)

,0,1,2,3,4,5,6,7,8,product
144,0.000000,0.0,0.0,0.413487,0.0,0.0,0.0,0.0,0.0,0.0
224,0.371842,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [138]:
# Alternative approach guided-LDA
# reference links below
#1) https://medium.freecodecamp.org/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164
#2)https://github.com/vi3k6i5/GuidedLDA

import numpy as np
import guidedlda
model = guidedlda.GuidedLDA(n_topics=15, n_iter=4000, random_state=7, refresh=20)

ModuleNotFoundError: No module named 'guidedlda'

In [69]:
# Document-Term Matrix-(count matrix)
vectorizer2 = CountVectorizer(encoding='utf-8',stop_words=None,max_df=0.997, min_df=0.001, binary=False,lowercase=True)
doc_word_mat2 = vectorizer2.fit_transform(reset_data.text_data)
doc_word_mat2 = doc_word_mat2.toarray()

In [70]:
# Get words that label the columns (needed to extract readable topics and make anchoring easier)
words2 = list(np.asarray(vectorizer2.get_feature_names()))
word2id = dict((v, idx) for idx, v in enumerate(words2))

In [71]:
# seeding anchor words
seed_topic_list = [["hoongee","hongee","oolaa"],["galaana","galaanaa","guutuu","lolaa"],
                ["balaa","dhibee","irraa"],
                ["sochii","lafaa"]]

seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

In [73]:
model.fit(doc_word_mat2, seed_topics=seed_topics, seed_confidence=0.15)

INFO:guidedlda:n_documents: 5871
INFO:guidedlda:vocab_size: 8360
INFO:guidedlda:n_words: 442803
INFO:guidedlda:n_topics: 15
INFO:guidedlda:n_iter: 4000
INFO:guidedlda:<0> log likelihood: -4970386
INFO:guidedlda:<20> log likelihood: -3695558
INFO:guidedlda:<40> log likelihood: -3595373
INFO:guidedlda:<60> log likelihood: -3555486
INFO:guidedlda:<80> log likelihood: -3535340
INFO:guidedlda:<100> log likelihood: -3521209
INFO:guidedlda:<120> log likelihood: -3513010
INFO:guidedlda:<140> log likelihood: -3504078
INFO:guidedlda:<160> log likelihood: -3496875
INFO:guidedlda:<180> log likelihood: -3493097
INFO:guidedlda:<200> log likelihood: -3488878
INFO:guidedlda:<220> log likelihood: -3485970
INFO:guidedlda:<240> log likelihood: -3486597
INFO:guidedlda:<260> log likelihood: -3481151
INFO:guidedlda:<280> log likelihood: -3480183
INFO:guidedlda:<300> log likelihood: -3479413
INFO:guidedlda:<320> log likelihood: -3478216
INFO:guidedlda:<340> log likelihood: -3476232
INFO:guidedlda:<360> log l

INFO:guidedlda:<3420> log likelihood: -3441214
INFO:guidedlda:<3440> log likelihood: -3439796
INFO:guidedlda:<3460> log likelihood: -3440606
INFO:guidedlda:<3480> log likelihood: -3440446
INFO:guidedlda:<3500> log likelihood: -3440648
INFO:guidedlda:<3520> log likelihood: -3439044
INFO:guidedlda:<3540> log likelihood: -3441232
INFO:guidedlda:<3560> log likelihood: -3439744
INFO:guidedlda:<3580> log likelihood: -3440826
INFO:guidedlda:<3600> log likelihood: -3439982
INFO:guidedlda:<3620> log likelihood: -3439163
INFO:guidedlda:<3640> log likelihood: -3439646
INFO:guidedlda:<3660> log likelihood: -3439406
INFO:guidedlda:<3680> log likelihood: -3440281
INFO:guidedlda:<3700> log likelihood: -3440671
INFO:guidedlda:<3720> log likelihood: -3438571
INFO:guidedlda:<3740> log likelihood: -3439275
INFO:guidedlda:<3760> log likelihood: -3439291
INFO:guidedlda:<3780> log likelihood: -3441718
INFO:guidedlda:<3800> log likelihood: -3440252
INFO:guidedlda:<3820> log likelihood: -3438130
INFO:guidedld

In [74]:
doc_topic = model.doc_topic_
vlist = []
for i in range(len(doc_topic)):
    vlist.append(doc_topic[i].argmax())
vlist
index_list=np.where(np.asarray(vlist) == 0)[0].tolist()
print("0",len(index_list))
index_list.extend(np.where(np.asarray(vlist) == 1)[0].tolist())
print("1",len(index_list))
index_list.extend(np.where(np.asarray(vlist) == 2)[0].tolist())
print("2",len(index_list))
index_list.extend(np.where(np.asarray(vlist) == 3)[0].tolist())
print("3",len(index_list))
len(index_list)

0 116
1 164
2 826
3 1034


1034

In [77]:
temp1=reset_data.copy()

In [78]:
temp1.loc[index_list,'predicted_class_label']= "indomain"

temp1.loc[(temp1['predicted_class_label']!="indomain") , 'predicted_class_label'] = "nondomain"

In [79]:
# filter unknown class
print("shape before filtering unknown docs",temp1.shape )
temp1 = temp1.loc[temp1.class_type !="unk"]
print("shape after filtering unknown docs",temp1.shape )

shape before filtering unknown docs (5871, 5)
shape after filtering unknown docs (1970, 5)


In [80]:
from sklearn.metrics import confusion_matrix
cm=pd.DataFrame(confusion_matrix(temp1.class_type, temp1.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])
print(cm)

print("\n")
precision = cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

recall =  cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")
F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++          417         1389
Actual--           47          117


precision 0.8987068965517241 

recall 0.23089700996677742 

F1_score 0.36740088105726876 



In [81]:
# seeding anchor words
seed_topic_list2 = [["hoongee","hongee"],["galaana","galaanaa","guutuu"],
                ["balaa"]]

seed_topics2 = {}
for t_id, st in enumerate(seed_topic_list2):
    for word in st:
        seed_topics2[word2id[word]] = t_id

In [82]:
model.fit(doc_word_mat2, seed_topics=seed_topics2, seed_confidence=0.15)

INFO:guidedlda:n_documents: 5871
INFO:guidedlda:vocab_size: 8360
INFO:guidedlda:n_words: 442803
INFO:guidedlda:n_topics: 15
INFO:guidedlda:n_iter: 4000
INFO:guidedlda:<0> log likelihood: -4970962
INFO:guidedlda:<20> log likelihood: -3709338
INFO:guidedlda:<40> log likelihood: -3610146
INFO:guidedlda:<60> log likelihood: -3564880
INFO:guidedlda:<80> log likelihood: -3543168
INFO:guidedlda:<100> log likelihood: -3529117
INFO:guidedlda:<120> log likelihood: -3519373
INFO:guidedlda:<140> log likelihood: -3513058
INFO:guidedlda:<160> log likelihood: -3507347
INFO:guidedlda:<180> log likelihood: -3500740
INFO:guidedlda:<200> log likelihood: -3498525
INFO:guidedlda:<220> log likelihood: -3493081
INFO:guidedlda:<240> log likelihood: -3492280
INFO:guidedlda:<260> log likelihood: -3489956
INFO:guidedlda:<280> log likelihood: -3486265
INFO:guidedlda:<300> log likelihood: -3483813
INFO:guidedlda:<320> log likelihood: -3481032
INFO:guidedlda:<340> log likelihood: -3479070
INFO:guidedlda:<360> log l

INFO:guidedlda:<3420> log likelihood: -3448966
INFO:guidedlda:<3440> log likelihood: -3447994
INFO:guidedlda:<3460> log likelihood: -3448664
INFO:guidedlda:<3480> log likelihood: -3448371
INFO:guidedlda:<3500> log likelihood: -3446983
INFO:guidedlda:<3520> log likelihood: -3447237
INFO:guidedlda:<3540> log likelihood: -3448123
INFO:guidedlda:<3560> log likelihood: -3448172
INFO:guidedlda:<3580> log likelihood: -3447439
INFO:guidedlda:<3600> log likelihood: -3447539
INFO:guidedlda:<3620> log likelihood: -3447245
INFO:guidedlda:<3640> log likelihood: -3446818
INFO:guidedlda:<3660> log likelihood: -3446796
INFO:guidedlda:<3680> log likelihood: -3446901
INFO:guidedlda:<3700> log likelihood: -3447022
INFO:guidedlda:<3720> log likelihood: -3445407
INFO:guidedlda:<3740> log likelihood: -3445882
INFO:guidedlda:<3760> log likelihood: -3446438
INFO:guidedlda:<3780> log likelihood: -3446785
INFO:guidedlda:<3800> log likelihood: -3444795
INFO:guidedlda:<3820> log likelihood: -3446196
INFO:guidedld

In [88]:
doc_topic = model.doc_topic_
vlist = []
for i in range(len(doc_topic)):
    vlist.append(doc_topic[i].argmax())
vlist
index_list=np.where(np.asarray(vlist) == 0)[0].tolist()
print("0",len(index_list))
index_list.extend(np.where(np.asarray(vlist) == 1)[0].tolist())
print("1",len(index_list))
index_list.extend(np.where(np.asarray(vlist) == 2)[0].tolist())
print("2",len(index_list))
len(index_list)

0 105
1 179
2 333


333

In [89]:
temp2=reset_data.copy()

In [90]:
temp2.loc[index_list,'predicted_class_label']= "indomain"

temp2.loc[(temp2['predicted_class_label']!="indomain") , 'predicted_class_label'] = "nondomain"

In [91]:
# filter unknown class
print("shape before filtering unknown docs",temp2.shape )
temp2 = temp2.loc[temp2.class_type !="unk"]
print("shape after filtering unknown docs",temp2.shape )

shape before filtering unknown docs (5871, 5)
shape after filtering unknown docs (1970, 5)


In [92]:
from sklearn.metrics import confusion_matrix
cm=pd.DataFrame(confusion_matrix(temp2.class_type, temp2.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])
print(cm)

print("\n")
precision = cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

recall =  cm.loc["Actual++","Predicted++"]/(cm.loc["Actual++","Predicted++"] + cm.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")
F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++          108         1698
Actual--           13          151


precision 0.8925619834710744 

recall 0.059800664451827246 

F1_score 0.11209133367929423 



In [95]:
reset_data.shape

(5871, 4)

In [100]:
# Baseline 1

baseline_temp = reset_data
baseline_temp['count'] = baseline_temp['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'hoongee|hongee|oolaa|galaana|galaanaa|guutuu|lolaa|balaa|dhibee|irraa|sochii|lafaa').sum())

# assigning class lables based on the binary classification result
baseline_temp.loc[baseline_temp['count'] >0, 'predicted_class_label'] = "indomain"
baseline_temp.loc[baseline_temp['count'] <=0, 'predicted_class_label'] = "nondomain"
baseline_temp['class_type'].unique()

# filter unknown class

baseline_temp = baseline_temp.loc[baseline_temp.class_type !="unk"]



In [102]:

from sklearn.metrics import confusion_matrix
cf=pd.DataFrame(confusion_matrix(baseline_temp.class_type, baseline_temp.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])


print(cf,"\n")


precision = cf.loc["Actual++","Predicted++"]/(cf.loc["Actual++","Predicted++"] + cf.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

recall =  cf.loc["Actual++","Predicted++"]/(cf.loc["Actual++","Predicted++"] + cf.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++          292         1514
Actual--           29          135 

precision 0.9096573208722741 

recall 0.16168327796234774 

F1_score 0.27456511518570764 



In [104]:
# Baseline 2

baseline_temp2 = reset_data
baseline_temp2['count'] = baseline_temp2['text_data'].apply(lambda x: pd.Series(x.split()).str.contains(r'hoongee|hongee|galaana|galaanaa|guutuu|balaa').sum())

# assigning class lables based on the binary classification result
baseline_temp2.loc[baseline_temp2['count'] >0, 'predicted_class_label'] = "indomain"
baseline_temp2.loc[baseline_temp2['count'] <=0, 'predicted_class_label'] = "nondomain"
baseline_temp2['class_type'].unique()

# filter unknown class

baseline_temp2 = baseline_temp2.loc[baseline_temp2.class_type !="unk"]



In [105]:

from sklearn.metrics import confusion_matrix
cf=pd.DataFrame(confusion_matrix(baseline_temp2.class_type, baseline_temp2.predicted_class_label),columns=["Predicted++","Predicted--"],index=["Actual++","Actual--"])


print(cf,"\n")


precision = cf.loc["Actual++","Predicted++"]/(cf.loc["Actual++","Predicted++"] + cf.loc["Actual--","Predicted++"])
print("precision",precision,"\n")

recall =  cf.loc["Actual++","Predicted++"]/(cf.loc["Actual++","Predicted++"] + cf.loc["Actual++","Predicted--"] )
print("recall",recall,"\n")

F1_score = 2*(precision * recall)/(precision + recall)
print("F1_score",F1_score,"\n")

          Predicted++  Predicted--
Actual++          109         1697
Actual--           11          153 

precision 0.9083333333333333 

recall 0.06035437430786268 

F1_score 0.11318795430944964 

